In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time
import statistics
import sklearn
%matplotlib inline
from datetime import date
from scipy import stats as stats
from sklearn import preprocessing
from sklearn import model_selection
from datetime import datetime, timedelta
from tensorflow import keras as keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from numpy import asarray
from numpy import savetxt

In [2]:
df_all = pd.read_csv("/Users/alex/df_all.csv")

In [3]:
df_all.columns

Index(['Unnamed: 0', 'HeartRate', 'SystolicBP', 'Potassium', 'Temperature',
       'White_Blood_Cell', 'Sodium', 'Bicarbonate', 'Glasgow_Coma',
       'Birillium', 'BloodUreaNitrogen', 'Urine', 'Mech_Vent', 'HeartRateMask',
       'SystolicBPMask', 'PotassiumMask', 'TemperatureMask',
       'White_Blood_CellMask', 'SodiumMask', 'BicarbonateMask',
       'Glasgow_ComaMask', 'BirilliumMask', 'BloodUreaNitrogenMask',
       'UrineMask', 'Mech_VentMask', 'HADM_ID',
       'blood and blood-forming organs', 'circulatory', 'congenital',
       'digestive', 'endocrine, metabolic and immunity',
       'external causes of injury and supplemental classification',
       'genitourinary', 'ill-defined', 'infectious and parasitic',
       'injury and poisoning', 'mental', 'musculoskeletal', 'neoplasms',
       'nervous', 'perinatal period', 'pregnancy, childbirth, and puerperium',
       'respiratory', 'skin and subcutaneous tissue', 'EMERGENCY', 'URGENT',
       'AGE BIN', 'Age Bin_2', 'Age Bin_3',

In [ ]:
#Make a table of each vital with its mask
HR = df_all[['HADM_ID','HeartRate', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'HeartRateMask']]
BP = df_all[['HADM_ID','SystolicBP', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'SystolicBPMask']]
Pot = df_all[['HADM_ID','Potassium', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'PotassiumMask']]
Temp = df_all[['HADM_ID','Temperature', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'TemperatureMask']]
WBC = df_all[['HADM_ID','White_Blood_Cell', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'White_Blood_CellMask']]
Na = df_all[['HADM_ID','Sodium', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'SodiumMask']]
Bi = df_all[['HADM_ID','Bicarbonate', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BicarbonateMask']]
Glas = df_all[['HADM_ID','Glasgow_Coma', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'Glasgow_ComaMask']]
Biril = df_all[['HADM_ID','Birillium', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BirilliumMask']]
BUN = df_all[['HADM_ID','BloodUreaNitrogen', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BloodUreaNitrogenMask']]
Urine = df_all[['HADM_ID','Urine', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'UrineMask']]
Mech_Vent = df_all[['HADM_ID','Mech_Vent', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'Mech_VentMask']]

In [ ]:
#Create a list to make columns for timeframe
names = []
for i in range(0, 336):
    j = str(i)
    names.append(j)   

In [91]:
#Makes y for prediction
def makeanswer(data):
    length = 336
    hadmlist = data['HADM_ID'].unique()
    timeline = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    timeline['HADM_ID'] = hadmlist
    y = []
    for i in range(0, len(hadmlist)):
        df = data[data['HADM_ID'] == hadmlist[i]]
        lst = []
        counter = 0
        for j in range(1, (len(df)-1)):
            pos = int(df.iloc[j, 2])
            left = int(df.iloc[j, 3])
            if pos > 335:
                pass
            else:
                lst.append(left)
        y.append(lst)
    
    return y

In [92]:
answers = makeanswer(HR)

In [93]:
len(answers)

22404

In [95]:
y = pd.DataFrame(answers)
y.to_csv(r'/Users/alex/SAPSy.csv')

In [ ]:
def makeset(data, value):
    length = 336
    hadmlist = data['HADM_ID'].unique()
    timeline = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    mask = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    timeline['HADM_ID'] = hadmlist
    mask['HADM_ID'] = hadmlist
    cols = []
    c = []
    y = []
    percent = 0
    for i in range(0, len(hadmlist)-1):
        if (i % 224 ) == 0:
            percent += 1
            print(str(percent) + '% done.') 
        df = data[data['HADM_ID'] == hadmlist[i]]
        lst = []
        counter = 0
        for j in range(1, (len(df)-1)):
            timein = int(df.iloc[j, 4])
            value = int(df.iloc[j, 1])
            pos = int(df.iloc[j, 2])
            left = int(df.iloc[j, 3])
            if pos > 335:
                pass
            else:
                timeline.iloc[i, pos] = value
                mask.iloc[i, pos] = 1
                lst.append(left)
            if timein < length:
                timeline.iloc[i, timein] = -1

        y.append(lst)
        
    timeline = timeline.iloc[:, 1:]
    mask = mask.iloc[:, 1:]
    mask = mask.fillna(0)
    timeline = timeline.ffill(axis = 1)
    timeline = timeline.fillna(value=value)
    
    return timeline, mask, y

In [ ]:
def makechannel(data, name):
    channel = pd.DataFrame()
    for i in range(0, 336):
        j = str(i)
        mname = ('m' + name + j)
        tname = ('t'+ name + j)
        channel[tname] = data[0].iloc[:, i]
        channel[mname] = data[1].iloc[:,i]
    return channel

In [ ]:
df_demo = pd.read_csv("/Users/alex/df_sapsdemo.csv")

In [ ]:
df_demo.columns

In [ ]:
hadmlist = HR['HADM_ID'].unique()
hadmlist = pd.DataFrame(hadmlist)
hadmlist['HADM_ID'] = hadmlist[0]

In [ ]:
demodata = pd.merge(hadmlist, df_demo, on='HADM_ID')

In [ ]:
demodata.shape

In [ ]:
demodata = demodata.drop_duplicates(subset = ['HADM_ID'])

In [ ]:
demodata.shape

In [ ]:
missing = set(hadmlist['HADM_ID']).symmetric_difference(set(demodata['HADM_ID']))

In [ ]:
missing

In [ ]:
dfmissing = pd.DataFrame({'HADM_ID': [107486, 130888, 136883, 141974, 189968], 
                          'EMERGENCY': [0,0,0,0,0], 
                          'URGENT': [0,0,0,0,0], 
                          'Age Bin_1': [0,0,0,0,0],
                          'Age Bin_2': [0,0,0,0,0], 
                          'Age Bin_3': [0,0,0,0,0], 
                          'Age Bin_4': [0,0,0,0,0], 
                          'Age Bin_5': [0,0,0,0,0], 
                          'Age Bin_6': [0,0,0,0,0],
                          'infectious and parasitic': [0,0,0,0,0], 
                          'neoplasms': [0,0,0,0,0],
                          'blood and blood-forming organs': [0,0,0,0,0]})

In [ ]:
demodata = demodata.append(dfmissing)

In [ ]:
demodata.shape

In [ ]:
demodata = demodata[[ 'HADM_ID','EMERGENCY','URGENT','Age Bin_1', 'Age Bin_2','Age Bin_3','Age Bin_4','Age Bin_5',
                        'Age Bin_6','infectious and parasitic','neoplasms', 'blood and blood-forming organs']]

In [ ]:
demodata = pd.merge(hadmlist, demodata, on='HADM_ID')

In [ ]:
demodata.to_csv(r'/Users/alex/df_sapsdemo.csv')

In [ ]:
HR = makeset(HR, 86)
HRSave = makechannel(HR, 'HR')
HRSave.to_csv(r'/Users/alex/HR.csv')

In [ ]:
BP = makeset(BP, 120)
BPSave = makechannel(BP, 'BP')
BPSave.to_csv(r'/Users/alex/BP.csv')

In [ ]:
Pot = makeset(Pot, 4.3)
PotSave = makechannel(Pot, 'Pot')
PotSave.to_csv(r'/Users/alex/Pot.csv')

In [ ]:
Temp = makeset(Temp, 37.8)
TempSave = makechannel(Temp, 'Temp')
TempSave.to_csv(r'/Users/alex/Temp.csv')

In [ ]:
#WBC = makeset(WBC, 9)
#WBCSave = makechannel(WBC, 'WBC')
#WBCSave.to_csv(r'/Users/alex/WBC.csv')

In [ ]:
#Na = makeset(Na, 140)
#NaSave = makechannel(Na, 'Na')
#NaSave.to_csv(r'/Users/alex/Na.csv')

In [ ]:
#Bi = makeset(Bi, 26)
#BiSave = makechannel(Bi, 'Bi')
#BiSave.to_csv(r'/Users/alex/Bi.csv')

In [ ]:
#Glas = makeset(Glas, 3)
#GlasSave = makechannel(Glas, 'Glas')
#GlasSave.to_csv(r'/Users/alex/Glas.csv')

In [ ]:
Mech_Vent = makeset(Mech_Vent, 0)
MechSave = makechannel(Mech_Vent, 'Mech_Vent')
MechSave.to_csv(r'/Users/alex/Mech_Vent.csv')

In [ ]:
Biril = makeset(Biril, 0.2)
BirilSave = makechannel(Biril, 'Biril')
BirilSave.to_csv(r'/Users/alex/Biril.csv')

In [ ]:
Urine = makeset(Urine, 150)
UrineSave = makechannel(Urine, 'Urine')
UrineSave.to_csv(r'/Users/alex/Urine.csv')

In [ ]:
BUN = makeset(BUN, 4.7)
BUNSave = makechannel(BUN, 'BUN')
BUNSave.to_csv(r'/Users/alex/BUN.csv')

In [ ]:
def makey(data):
    y = data[2]
    return y

y = makey(HR)
y = pd.DataFrame(y)
y.to_csv(r'/Users/alex/SAPSy.csv')

In [ ]:
lst = [HR, BP, Pot, Temp, WBC, Na, Bi, Glas, Mech_Vent, Biril, Urine, BUN]
lstdata = ['HR', 'BP', 'Pot', 'Temp', 'WBC', 'Na', 'Bi', 'Glas', 'Mech_Vent', 'Biril', 'Urine', 'BUN']
demo = demodata
lstdemo = ['Age1' , 'Age2', 'Age3', 'Age4', 'Age5', 'AIDS', 'Hema', 'Cancer', admissions here]

def makefinal(lst, lstdata, demo, lstdemo):
    binary = pd.DataFrame()
    for i in range(0, 336):
        j = str(i)
        for k in range(0, len(lstdata)):
            mname = ('m' + str(lstdata[k]) + j)
            tname = ('t'+ lstdata[k] + j)
            
            binary[tname] = lst[k][0].iloc[:, i]
            binary[mname] = lst[k][1].iloc[:, i]
        for l in range(0, len(lstdemo)):
            dname = (lstdemo[l] + j)
            binary[dname] = demo.iloc[:, l]
        
    return binary

In [ ]:
def makechannel(data, name):
    channel = pd.DataFrame()
    for i in range(0, 336):
        j = str(i)
        mname = ('m' + name + j)
        tname = ('t'+ name + j)
        channel[tname] = data[0].iloc[:, i]
        channel[mname] = data[1].iloc[:,i]
    return binary

In [ ]:
binary = makefinal(lst, lstdata)
binary = np.asarray(binary)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(binary, test_size=0.15, shuffle = False)

print(train.shape[0])
print(test.shape[0])

scaler = StandardScaler()

trnx = scaler.fit_transform(train)
tstx = scaler.transform(test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
y = pad_sequences(y, padding= 'post' )
trny = y[:850]
tsty = y[850:]

In [ ]:
y.shape

In [ ]:
trny.shape

In [ ]:
trnx.shape

In [ ]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# save to csv file
savetxt('binSAPSIItrnx.csv', trnx, delimiter=',')
savetxt('binSAPSIItrny.csv', trny, delimiter=',')
savetxt('binSAPSIItstx.csv', tstx, delimiter=',')
savetxt('binSAPSIItsty.csv', tsty, delimiter=',')

In [ ]:
trnx = trnx.reshape((trnx.shape[0], 336, 24))
tstx = tstx.reshape((tstx.shape[0], 336, 24))

In [ ]:
trnx.shape

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow
from sklearn import preprocessing
from sklearn import model_selection
from tensorflow import keras as keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from numpy import loadtxt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate, Dense, LSTM, Input, concatenate, Dropout, TimeDistributed
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

tensorflow.keras.backend.set_floatx('float64')

cells = 128

inptrnx = Input(shape=(trnx.shape[1], trnx.shape[2]))

model = Sequential()
model.add(LSTM(cells, return_sequences = True, input_shape=(trnx.shape[1], trnx.shape[2])))
model.add(LSTM(cells, return_sequences = True))
model.add(TimeDistributed(Dense(1, activation = 'relu')))
model.compile(optimizer= 'adam' , loss= 'mean_squared_error' , 
                metrics=['mean_absolute_percentage_error'])

#MAD, MAPE, MSE, MSLE, R2, Kappa 
history = model.fit(trnx, trny, 
                epochs=8, batch_size = 128, validation_split=0.1, verbose=2)

print("Evaluate on test data")
results = model.evaluate(tstx, tsty, batch_size = 128)
print("MSE, ACC, MAE:", results)



# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()